In [ ]:
import pickle
import numpy as np  
import pandas as pd

import drive.MyDrive.preprocess_tools as pts

!git clone https://github.com/ArnaudFRANCOISE/StackOverflow_tags_prediction.git
!ls

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
fatal: destination path 'StackOverflow_tags_prediction' already exists and is not an empty directory.
drive  sample_data  StackOverflow_tags_prediction


In [ ]:
import os
import pandas as pd

from os import listdir
from os.path import isfile, join

mypath = "/content/StackOverflow_tags_prediction/QueryResults/QueryResults/"

filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#gather all csv into one dataframe
#print(len(filenames))
data = pd.read_csv(mypath+filenames[0])
data

for filename in filenames[1:]:
  d = pd.read_csv(mypath+filename)
  data = pd.concat([data, d.iloc[:5000,:]], axis=0)
  #print(data.shape)

#eliminate duplicate
data = data.drop_duplicates(subset=['Body'])
data

,Id,Body,Title,Tags,CreationDate
0,51964879,<p>I am learning to write the groovy script to...,Configure matrix authorization plugin using Gr...,<jenkins><groovy><jenkins-plugins><jenkins-gro...,2018-08-22 10:31:25
1,62701591,"<p>New to spring magic, hoping spring magic ca...",Forcing Spring to call subclass for one method,<spring><spring-boot><groovy><annotations><spr...,2020-07-02 17:07:15
2,45597662,<p>Trying to post jenkins log to rest end poin...,parse jenkins job to get last few lines and pa...,<jenkins><groovy>,2017-08-09 17:59:54
3,19803888,<p>I want to get application path in my applic...,How to get app path in grails?,<grails><groovy><grails-2.0>,2013-11-06 04:03:26
4,3402152,"<p>Coming from a J2EE dev background, I love t...",Is Grails a viable option for large scale ente...,<java><grails><jakarta-ee><groovy>,2010-08-04 02:23:05
...,...,...,...,...,...
4995,11257201,<p>i want to display more than one images in a...,Display more than one images from drawable fol...,<android>,2012-06-29 06:41:58
4996,54072808,"<p>I created a <a href=""https://developers.goo...",Using custom google action offline on a smartp...,<android><google-assistant-sdk>,2019-01-07 10:44:26
4997,11257210,<p>I have two buttons in android. Btn1 and Btn...,Button click action in android?,<android>,2012-06-29 06:43:13
4998,3567420,<p>I am trying to write an application that wi...,How to catch that map panning and zoom are rea...,<android><google-maps><drag><dynamic-data>,2010-08-25 15:15:30


In [ ]:
import numpy as np 
import pandas as pd 
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [ ]:
from keras.preprocessing.text import Tokenizer
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Body'].values)
X = tokenizer.texts_to_sequences(data['Body'].values)




In [ ]:
X = pad_sequences(X)

In [ ]:
X

array([[  0,   0,   0, ..., 555, 191,   1],
       [  0,   0,   0, ..., 842, 571,   1],
       [  0,   0,   0, ..., 860, 312,   1],
       ...,
       [  0,   0,   0, ...,  11, 559,   1],
       [  0,   0,   0, ...,  17,  12,   1],
       [  0,   0,   0, ...,  64, 305,   1]], dtype=int32)

In [ ]:
type(X)
X = pd.DataFrame(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133283 entries, 0 to 133282
Columns: 6203 entries, 0 to 6202
dtypes: int32(6203)
memory usage: 3.1 GB


In [ ]:
  """pickle_out = open("/content/drive/MyDrive/Etudes/IA/stackoverflow_/y_lstmrawtags.pickle","wb")
  pickle.dump(data["Tags"], pickle_out)
  pickle_out.close() """

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, data['Tags'], test_size=0.20, random_state=42)

In [ ]:

import pickle
def save():
  pickle_out = open("/content/drive/MyDrive/Etudes/IA/stackoverflow_/X_train_seq.pickle","wb")
  pickle.dump(X_train, pickle_out)
  pickle_out.close()

  pickle_out = open("/content/drive/MyDrive/Etudes/IA/stackoverflow_/X_test_seq.pickle","wb")
  pickle.dump(X_test, pickle_out)
  pickle_out.close()


save()

In [ ]:

target_interest  = ["Assembly", "C", "C++", "CoffeeScript", "Go", "Groovy", "Haskell", "Java", 
                    "JavaScript", "Lua", "Matlab", "Objective-c","html","css","android","Perl", "PHP",
                    "Python", "R", "Ruby", "Scala", "SQL", "Swift", "TypeScript","Vb.Net", "Vba"]
target_interest = list(str_.lower() for str_ in target_interest)
print(len(target_interest))
print(target_interest)

26
['assembly', 'c', 'c++', 'coffeescript', 'go', 'groovy', 'haskell', 'java', 'javascript', 'lua', 'matlab', 'objective-c', 'html', 'css', 'android', 'perl', 'php', 'python', 'r', 'ruby', 'scala', 'sql', 'swift', 'typescript', 'vb.net', 'vba']


In [ ]:
def clean_tags(tags_content):
  cleaned = []
  
  
  token_vec = pts.tokenizer(tags_content)
  wrong = ['>','<']
  for tag in token_vec:
    if tag not in wrong:
      cleaned.append(tag)
  return cleaned

print(clean_tags(y_train.iloc[0]))

['php', 'mysql']


In [ ]:
#10min
def target_selection_to_bin(y_train, y_test):
  y_train, y_test
  target_train = np.zeros((y_train.shape[0], 26))

  target_test = np.zeros((y_test.shape[0], 26))
  len1, len2 = y_train.shape[0], y_test.shape[0]

  for i in range(y_train.shape[0]):
    #print(data["Tags"].iloc[i])
    tag_list  = clean_tags(y_train.iloc[i])
    #print(tag_list)
    for tag_ in tag_list:
      tag = str(tag_)

      if (tag in target_interest):
        print("Present tag: ",tag,"progress: ", np.floor(i/len1*100),'% ',end="\r")

        target_train[i,target_interest.index(tag)] = 1
        #print(data["Tags"].iloc[i])

  for i in range(y_test.shape[0]):
    #print(data["Tags"].iloc[i])
    tag_list  = clean_tags(y_test.iloc[i])
    #print(tag_list)
    for tag_ in tag_list:
      tag = str(tag_)

      if (tag in target_interest):
        print("Present tag: ",tag,"progress: ", np.floor(i/len2*100),'% ',end="\r")

        target_test[i,target_interest.index(tag)] = 1
        #print(data["Tags"].iloc[i])
  return target_train, target_test

y_train_bin, y_test_bin = target_selection_to_bin(y_train, y_test)

In [ ]:
pickle_out = open("/content/drive/MyDrive/Etudes/IA/stackoverflow_/target_test_lstm.pickle","wb")
pickle.dump(y_train_bin, pickle_out)
pickle_out.close()
pickle_out = open("/content/drive/MyDrive/Etudes/IA/stackoverflow_/target_ltsm.pickle","wb")
pickle.dump(y_test_bin , pickle_out)
pickle_out.close()

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(106626, 6203)
(26657, 6203)
(106626,)
(26657,)


In [ ]:
#label distribution
print("data distribution test set  \t train set")
for i in range(len(target_interest)):
  print(target_interest[i])
  print("\t\t",np.count_nonzero(y_test_bin[:,i] == 1, axis=0),'\t\t', np.count_nonzero(y_train_bin[:,i] == 1,axis=0))

data distribution test set  	 train set
assembly
		 970 		 4033
c
		 2427 		 9610
c++
		 1258 		 4958
coffeescript
		 998 		 4002
go
		 997 		 4009
groovy
		 1997 		 8003
haskell
		 969 		 4037
java
		 1578 		 6355
javascript
		 2119 		 8483
lua
		 972 		 4028
matlab
		 1014 		 3990
objective-c
		 1000 		 4093
html
		 1290 		 5329
css
		 994 		 4172
android
		 1106 		 4235
perl
		 1038 		 3968
php
		 1161 		 4539
python
		 1188 		 4640
r
		 1015 		 4029
ruby
		 1050 		 4145
scala
		 1020 		 4037
sql
		 1056 		 4395
swift
		 1029 		 3981
typescript
		 995 		 4023
vb.net
		 965 		 4042
vba
		 1030 		 3990


In [ ]:
print(target.shape)
print(x_train.shape)

(256352, 26)
(256352, 40000)
